# LANE DETECTION



This project is focused on detecting road lane lines using a combination of YOLO object detection and Hough Line Transform techniques. The goal is to create a lane detection system that combines the strengths of both methods for better accuracy, especially in detecting vertical lane lines (which are crucial for navigation and driving assistance systems)

YOLO is not the first model I tried to integrate, previously it was LaneNet which was more specialized in line detection, but it was a completely unsuccessful experience. The pre-trained YOLO model (on TuSimple, CULane datasets) seemed redundant, as it involves not only lane/line detection, but also detection of many other road objects. Training my own fine-tuned lane/line weights turned out to be too time-consuming, so I settled on the pre-trained version.

The model itself is not ideal in low light conditions or, as in the input example, with sunset conditions. So the idea was to improve lane detection based on Hough Line Transform line detection. Theoretically, the combination of methods should improve lane detection to allow the driving assistance system to better understand the vehicle’s position.

Inputs:
- Video file
- YOLO Model (lane_trained.pt): pre-trained model is used to detect lane-related objects (such as lane lines, arrows, and other road markings)

Output:
- Annotated Frames and Lane Lines
- Bounding Boxes: YOLO detects lane-related objects and draws bounding boxes around them
- Vertical Lane Lines: Detected using Hough Line Transform, filtered for verticality
- Annotations: Labels indicating lane types, e.g., ‘Line 1’, ‘Yellow Markings’

Advantages:
1. Combining YOLO and Hough Transform:
- YOLO is good in detecting objects (lane markings, arrows, etc.), which provides a great initial step in lane detection. It locates regions of interest quickly and with good accuracy.
- Hough Line Transform is a robust method for detecting lines, particularly vertical lines, which are the most important for lane tracking. It should work well even in noisy or complex environments.
2. Filtering Horizontal Lines: By filtering out horizontal lines (which are irrelevant for lane detection), we focus only on vertical lane lines, improving the quality of the lane markings detected.
3. Efficient and Fast: This method uses YOLO for quick object detection followed by Hough Transform for precise line detection. It’s a hybrid approach that leverages the best of both techniques.
4. Adaptability: The solution can easily be adapted for different road types, lighting conditions, and lane configurations by fine-tuning the YOLO model.

Disadvantages:
1. Quality of YOLO Model: the project relies on the pre-trained YOLO model. The model requires better fine tuniing and to be specialised on detection of road lines only.
2. Limited Detection of Curved Lanes: YOLO and Hough Lines are generally optimized for straight lines, so detecting curved lanes is more challenging.
3. Complexity of Pipeline: The system combines two methods, which increases complexity. Maintaining and fine-tuning the pipeline might require extra efforts.
4. False Positives/Negatives: In crowded/noicy scenes (e.g with many vehicles), the YOLO model might generate false positives (e.g. detecting other objects as lane lines), and Hough might struggle to detect lines in highly cluttered environments.

Result:
 At this point the result isn't quite great. The project requires extra tunning to improve lines detection in general, remove noicy output and better approach to draw lines for dashed lines.

In [1]:
from ultralytics import YOLO
import cv2
import numpy as np

# Paths
MODEL_PATH = 'yolov8n.pt'
VIDEO_PATH = 'sunset.mp4'

In [ ]:
# Pre-trained yolo model, my attempts to train it on TuSimple data set failed due to luck of resources and time
lane_model = YOLO('lane_trained.pt')

# Lane classes
lane_class_indices = [0, 1, 2, 3]  # extract line based classes, change depending on trained dataset

"""
Extract lane-related points from YOLO detections and filter out horizontal lines
"""
def process_yolo_detections(frame, results):
    left_lane_points = []
    right_lane_points = []

    for box in results[0].boxes.data:
        x1, y1, x2, y2, conf, cls = box
        cls = int(cls)

        # Filter only lane-related classes
        if cls in lane_class_indices:
            # Calculate the center of the bounding box
            box_center_x = (x1 + x2) / 2
            box_center_y = (y1 + y2) / 2

            # Filter out horizontal lines
            slope = (y2 - y1) / (x2 - x1 + 1e-6)
            if abs(slope) < 0.5:  # Adjust the threshold for filtering horizontal lines
                continue  # Skip this bounding box if it's horizontal

            # Assign to left or right lane based on x-position
            if box_center_x < frame.shape[1] // 2:
                left_lane_points.append(((x1 + x2) // 2, y2))
            else:
                right_lane_points.append(((x1 + x2) // 2, y2))

    return left_lane_points, right_lane_points

"""
Fit a line through points and draw it on the frame.
"""
def draw_lane_line(points, color, frame):
    if len(points) > 1:
        points = np.array(points, dtype=np.int32)
        [vx, vy, x, y] = cv2.fitLine(points, cv2.DIST_L2, 0, 0.01, 0.01)
        slope = vy / vx
        intercept = y - slope * x

        # Define the start and end points for the line
        y1 = frame.shape[0]
        y2 = int(frame.shape[0] * 0.6)
        x1 = int((y1 - intercept) / slope)
        x2 = int((y2 - intercept) / slope)

        cv2.line(frame, (x1, y1), (x2, y2), color, 2)

"""
Detect and draw additional lane lines using Hough Transform.
"""
def detect_hough_lines(frame, results):
    # prepare teh frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    #Mask ROI
    mask = np.zeros_like(edges)
    height, width = frame.shape[:2]
    polygon = np.array([[
        (0, height),
        (width, height),
        (width // 2 + 100, height // 2),
        (width // 2 - 100, height // 2),
    ]], dtype=np.int32)
    cv2.fillPoly(mask, polygon, 255)
    roi_edges = cv2.bitwise_and(edges, mask)

    for box in results[0].boxes.data:
        x1, y1, x2, y2, _, _ = box
        cv2.rectangle(roi_edges, (int(x1), int(y1)), (int(x2), int(y2)), 0, -1)

    # Hough Line Transform
    lines = cv2.HoughLinesP(roi_edges, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=10)

    # Return Detected Lines
    hough_lines = []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            # Filter out horizontal lines by checking the angle of the line
            slope = (y2 - y1) / (x2 - x1 + 1e-6)  # Avoid division by zero
            angle = np.arctan(slope) * 180 / np.pi  # Convert slope to angle in degrees

            # Filter out horizontal lines: we want lines with angles close to vertical
            if abs(angle) < 10 or abs(angle) > 170:  # For vertical lane lines, ignore near-horizontal lines
                continue

            hough_lines.append(((x1, y1), (x2, y2)))

    return hough_lines

"""
Draw Hough lines on the frame.
"""
def draw_hough_lines(hough_lines, frame):

    for line in hough_lines:
        (x1, y1), (x2, y2) = line
        color = (255, 0, 255) if x1 < frame.shape[1] // 2 else (0, 255, 255)  # Left magenta, Right yellow
        cv2.line(frame, (x1, y1), (x2, y2), color, 2)

def lane_detection_pipeline(original_frame):
    # Step 1: Make a copy for processing
    processing_frame = original_frame.copy()

    # Step 2: Run YOLO Model
    results = lane_model(processing_frame)

    # Step 3: Process YOLO Detections and Draw Boxes
    left_lane_points, right_lane_points = process_yolo_detections(processing_frame, results)
    
    # Draw YOLO Bounding Boxes
    for box in results[0].boxes.data:
        x1, y1, x2, y2, _, cls = box
        cls = int(cls)
        color = (255, 0, 0) if cls in lane_class_indices else (0, 255, 0)  # Blue for lanes, Green for others
        cv2.rectangle(processing_frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        cv2.putText(processing_frame, lane_model.names[cls], (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Step 4: Detect Hough Lines
    hough_lines = detect_hough_lines(original_frame.copy(), results)

    # Step 5: Make a copy for visualization
    visualization_frame = processing_frame.copy()

    # Step 6: Draw YOLO and Hough results
    draw_lane_line(left_lane_points, (255, 0, 0), visualization_frame)  # Blue for left lane
    draw_lane_line(right_lane_points, (0, 0, 255), visualization_frame)  # Red for right lane
    draw_hough_lines(hough_lines, visualization_frame)

    return visualization_frame

# Main loop for testing
cap = cv2.VideoCapture(VIDEO_PATH)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Apply lane detection pipeline
    output_frame = lane_detection_pipeline(frame)

    # Display the result
    cv2.imshow('Lane Detection', output_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[ WARN:0@0.535] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@0.535] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin0 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@0.535] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@0.535] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/c


0: 640x384 (no detections), 166.7ms
Speed: 1.9ms preprocess, 166.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 164.4ms
Speed: 7.0ms preprocess, 164.4ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 152.1ms
Speed: 1.2ms preprocess, 152.1ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 152.9ms
Speed: 1.2ms preprocess, 152.9ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 153.3ms
Speed: 1.1ms preprocess, 153.3ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 151.9ms
Speed: 1.4ms preprocess, 151.9ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 157.0ms
Speed: 1.2ms preprocess, 157.0ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 162.1ms
Speed: 1.6ms prepr